In [1]:
import sys
sys.path.append('../Utils')

In [2]:
import nltk
import networkx as nx
from collections import defaultdict
from nltk import word_tokenize
from nltk.util import ngrams
import spacy
nltk.download('punkt')
!python -m spacy download es_core_news_sm
import numpy as np
from bokeh.io import output_notebook, show
from bokeh.models import GraphRenderer, StaticLayoutProvider, Circle, MultiLine
from bokeh.plotting import figure
import networkx as nx
import pandas as pd

import torch
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/juandanielcastrellon/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 9.6 MB/s eta 0:00:00:00:0100:01

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


/Users/juandanielcastrellon/Documents/Universidad/CatorceavoSemestre/ProyGrado/Repo/tesis/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from Predict import get_sentiment
from Models import SentimentAnalysisPretrainedBert
from Constants import SEED

In [4]:
# Load the Spanish language model
nlp = spacy.load("es_core_news_sm")

def plural_to_singular(word):
    # Process the word using Spacy
    doc = nlp(word)
    # Return the lemma of the first token
    return doc[0].lemma_

CATEGORIES_DICT = {
    0: 'NEGATIVO',
    1: 'NEUTRO',
    2: 'POSITIVO',
}

#Modelo bert a usar
BERT_MODEL = '../../TonysModels/old-spanish-beto-base-cased.pt'

#Dispositivo sobre el que se corre el modelo de ML
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer_cased = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")

torch.manual_seed(SEED)

model = SentimentAnalysisPretrainedBert(BERT_MODEL, 0.4, 4, 256, 128,  True).to(DEVICE)

# Cargar un modelo que ya había sido Finetuneado
checkpoint_classifier = torch.load(
    "../BestModel/Mejores_guardados_beto_cased.pt",
    map_location=torch.device(DEVICE)
)

model.load_state_dict(checkpoint_classifier['model_state_dict'])


<All keys matched successfully>

In [ ]:
data = pd.read_csv('../../Resultados/Beto_tagged.csv', sep=';')
columns_to_drop = data.filter(like='Unnamed').columns
data = data.drop(columns=columns_to_drop)
data.head()

,file,Año,Periódico,Ciudad,Texto,Sentimiento,predictions,negative_probs,neutral_probs,positive_probs,ironic_probs,Texto_Lematizado,Sustantivos
0,../../Datos/19th_century_Latam_Newspapers_1/PF...,1870,Los loros,Popayán,publicacion semanal ignoramos si sus restos f...,NEUTRO,1.0,9.996460,55.532199,33.521801,0.949539,"['publicacion', 'semanal', 'ignorar', 'resto',...","['restos', 'fin', 'soldados', 'puestos', 'comb..."
1,../../Datos/19th_century_Latam_Newspapers_1/PF...,1870,Los loros,Popayán,se presente en su oficina el que monto en la g...,NEUTRO,1.0,17.339712,63.219225,18.328328,1.112728,"['presentar', 'oficina', 'monto', 'gallina', '...","['oficina', 'monto', 'gallina', 'miedo', 'desp..."
2,../../Datos/19th_century_Latam_Newspapers_1/PF...,1870,Los loros,Popayán,"de baron de las tenazas, a don agustin castill...",POSITIVO,2.0,7.059140,42.583779,49.531049,0.826027,"['bar', 'tenaza', 'don', 'agustin', 'castillo'...","['tenazas', 'don', 'agustin', 'castillo', 'mar..."
3,../../Datos/19th_century_Latam_Newspapers_1/PF...,1870,Los loros,Popayán,"al articulo presente, en una oracion ferviente...",POSITIVO,2.0,6.416973,21.443181,71.021354,1.118488,"['articulo', 'presente', 'oracion', 'ferviente...","['articulo', 'oracion', 'satanas', 'jamas', 'i..."
4,../../Datos/19th_century_Latam_Newspapers_1/PF...,1870,Los loros,Popayán,quien es el que desde aca tantas noticias envi...,NEUTRO,1.0,11.500288,60.579348,27.018249,0.902115,"['aca', 'tanto', 'noticia', 'envia', 'adivinar...","['noticias', 'nadie', 'ano', 'plazo', 'bribona..."


In [ ]:
words = [
    'numero', 'doctor', 'gobierno', 'senor', 'ciudad', 'alli', 'ano', 'bogota', 
    'pais', 'nombre', 'hombre', 'patria', 'dios', 'hombres', 'dia', 'obra', 
    'anos', 'muerte', 'pueblo', 'colombia'
]

len(words)

20

In [ ]:
from tqdm import tqdm

def crear_grafo(texts, words_to_search, context=40, num_times=3):
    # Crear un grafo no dirigido
    G = nx.Graph()

    # Preprocesar las palabras para manejar plurales
    words = [plural_to_singular(w) for w in words_to_search]
    words_to_index = {word: i for i, word in enumerate(words)}
    index_to_words = {i: word for i, word in enumerate(words)}

    word_appearences = {}

    edge_sentiment = {}

    node_sentiment = {}
    for text in tqdm(texts):
        tokens = word_tokenize(text.lower())
        # Generar n-gramas de tamaño 'context'

        for i, token in enumerate(tokens):
            min_token = plural_to_singular(token)



            if min_token in words:
                context_min = max(0, i-context)
                context_max = min(len(tokens), i+context)

                context_window = tokens[context_min:context_max]
                sentiment = get_sentiment(model, ' '.join(context_window), tokenizer_cased)[1]
                for word in context_window:
                    min_word = plural_to_singular(word)
                    if min_word in words and min_word!=min_token:
                        if G.has_edge(words_to_index[min_token], words_to_index[min_word]):
                            G[words_to_index[min_token]][words_to_index[min_word]]['weight'] += 1
                        else:
                            G.add_edge(words_to_index[min_token], words_to_index[min_word], weight=1)
                            sent = edge_sentiment.get((min_token, min_word), {})
                            sent[sentiment] = sent.get(sentiment, 0) + 1
                            edge_sentiment[(min_token, min_word)] = sent

                word_appearences[words_to_index[min_token]] = word_appearences.get(words_to_index[min_token], 0) + 1

                if min_token in node_sentiment:
                    node_sentiment[words_to_index[min_token]][sentiment] += 1
                else:
                    node_sentiment[words_to_index[min_token]] = {k: 0 for k in CATEGORIES_DICT.values()}
                    node_sentiment[words_to_index[min_token]][sentiment] += 1

    nx.set_node_attributes(G, word_appearences, 'appearances')

    most_common_sent = {w: max(f, key=f.get) for w, f in node_sentiment.items()}
    nx.set_node_attributes(G, most_common_sent, 'sentiment')

    nx.set_node_attributes(G, index_to_words, 'word')
    for edge, sentiments in edge_sentiment.items():
        most_common_sentiment = max(sentiments, key=sentiments.get)
        G[words_to_index[edge[0]]][words_to_index[edge[1]]]['sentiment'] = most_common_sentiment

    # Filtrar arcos con peso menor a 'num_times'
    for u, v, data in list(G.edges(data=True)):
        G[u][v]['weight'] /= 2
        if data['weight'] < num_times:
            G.remove_edge(u, v)

    return G

G = crear_grafo(list(data.Texto), words, context = 40, num_times = 10)

print("Nodos:", G.nodes(data=True))
print("Arcos:", G.edges(data=True))

In [ ]:
print("Nodos:", len(G.nodes(data=True)))
print("Arcos:", len(G.edges(data=True)))

Nodos: 18
Arcos: 153


In [30]:
nx.write_graphml(G, "../grafo/grafo-prueba1-40_contexto-10rep.graphml")

In [31]:
G = nx.read_graphml("../grafo/grafo-prueba1-40_contexto-10rep.graphml")

In [9]:
print("Nodos:", len(G.nodes(data=True)))
print("Arcos:", len(G.edges(data=True)))

Nodos: 18
Arcos: 153


In [27]:
output_notebook()

Loading BokehJS ...

In [22]:
from bokeh.plotting import figure, from_networkx, show
from bokeh.models import Legend, LegendItem
from bokeh.models import ColumnDataSource, LabelSet

In [23]:
words_to_index = {word: i for i, word in enumerate(words)}
index_to_words = {i: word for i, word in enumerate(words)}

In [24]:
words_to_index

{'numero': 0,
 'doctor': 1,
 'gobierno': 2,
 'senor': 3,
 'ciudad': 4,
 'alli': 5,
 'ano': 6,
 'bogota': 7,
 'pais': 8,
 'nombre': 9,
 'hombre': 10,
 'patria': 11,
 'dios': 12,
 'hombres': 13,
 'dia': 14,
 'obra': 15,
 'anos': 16,
 'muerte': 17,
 'pueblo': 18,
 'colombia': 19}

In [29]:
def plot_interactive_graph_bokeh(G, node, depth, size_scale=20, edge_scale=3):

    # Extraer el número de apariciones y el sentimiento de los nodos
    appearances = nx.get_node_attributes(G, 'appearances')
    node_sentiments = nx.get_node_attributes(G, 'sentiment')

    # Definir el mapeo de colores para los sentimientos en formato hexadecimal
    sentiment_colors = {
        'POSITIVO': '#4CAF50',  # Verde suave
        'NEGATIVO': '#F44336',  # Rojo suave
        'NEUTRO': '#9E9E9E',    # Gris suave
    }

    # Obtener los nodos que están a una distancia máxima de 'depth' del nodo en cuestión
    nodes_at_depth = nx.single_source_shortest_path_length(G, node, cutoff=depth).keys()
    subgraph = G.subgraph(nodes_at_depth)

    # Tamaño del gráfico en pulgadas
    width_in_inches = 8
    height_in_inches = 8
    # Convertir a píxeles
    width_in_pixels = width_in_inches * 96
    height_in_pixels = height_in_inches * 96


    # Crear la figura
    p = figure(x_range=(-2, 2), y_range=(-2, 2),
               title="Graph Interaction Demo",
               x_axis_location=None, y_axis_location=None,
               tools="hover,pan,wheel_zoom,reset", tooltips="appearances: @appearances sentiment: @sentiment",
               toolbar_location="above",
               width=width_in_pixels, height=height_in_pixels)
    p.grid.grid_line_color = None

    # Configurar el GraphRenderer
    graph = from_networkx(subgraph, nx.spring_layout, scale=2, center=(0, 0))
    p.renderers.append(graph)

    # Establecer los datos de los nodos y las aristas
    graph.node_renderer.data_source.data['colors'] = [sentiment_colors.get(node_sentiments[node]) for node in subgraph.nodes()]
    graph.node_renderer.data_source.data['size'] = [size_scale * appearances[node] for node in subgraph.nodes()]
    graph.edge_renderer.data_source.data['colors'] = [sentiment_colors.get(G[u][v]['sentiment'], 'black') for u, v in subgraph.edges()]
    graph.edge_renderer.data_source.data['width'] = [edge_scale * G[u][v]['weight'] for u, v in subgraph.edges()]

    graph.node_renderer.glyph.update(size='size', fill_color='colors')
    graph.edge_renderer.glyph.update(line_color='colors', line_width='width')


    x, y = zip(*graph.layout_provider.graph_layout.values())

    print(graph.node_renderer.data_source.data['word'])
    source = ColumnDataSource({'x':x,'y':y,'kid':[ix for ix in graph.node_renderer.data_source.data['word']]})
    labels = LabelSet(x='x', y='y', text='kid', source=source)

    p.renderers.append(labels)

    # Mostrar el gráfico
    show(p)

# Ejemplo de uso
plot_interactive_graph_bokeh(G, f'{words_to_index["bogota"]}', 2, size_scale=6, edge_scale=2)


['colombia', 'pueblo', 'doctor', 'senor', 'bogota', 'dio', 'hombre', 'obra', 'diar', 'numero', 'ciudad', 'muerte', 'ano', 'alli', 'gobierno', 'patria', 'pai', 'nombre']


In [18]:
G.nodes(data=True)

NodeDataView({'19': {'appearances': 2009, 'sentiment': 'NEGATIVO', 'word': 'colombia'}, '18': {'appearances': 3060, 'sentiment': 'NEGATIVO', 'word': 'pueblo'}, '1': {'appearances': 2455, 'sentiment': 'NEGATIVO', 'word': 'doctor'}, '3': {'appearances': 7906, 'sentiment': 'NEUTRO', 'word': 'senor'}, '7': {'appearances': 2382, 'sentiment': 'NEGATIVO', 'word': 'bogota'}, '12': {'appearances': 1858, 'sentiment': 'NEUTRO', 'word': 'dio'}, '13': {'appearances': 4404, 'sentiment': 'POSITIVO', 'word': 'hombre'}, '15': {'appearances': 2325, 'sentiment': 'NEGATIVO', 'word': 'obra'}, '14': {'appearances': 3270, 'sentiment': 'NEGATIVO', 'word': 'diar'}, '0': {'appearances': 2542, 'sentiment': 'NEUTRO', 'word': 'numero'}, '4': {'appearances': 2501, 'sentiment': 'NEGATIVO', 'word': 'ciudad'}, '17': {'appearances': 1522, 'sentiment': 'POSITIVO', 'word': 'muerte'}, '16': {'appearances': 4300, 'sentiment': 'NEGATIVO', 'word': 'ano'}, '5': {'appearances': 2137, 'sentiment': 'NEGATIVO', 'word': 'alli'}, '